# Parametros

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PATH_2210 = "Datos\E01_T_DEU_CONS_2210.csv"
PATH_2211 = "Datos\E01_T_DEU_CONS_2211.csv"
PATH_2212 = "Datos\E01_T_DEU_CONS_2212.csv"
PATH_2301 = "Datos\E01_T_DEU_CONS_2301.csv"
PATH_2302 = "Datos\E01_T_DEU_CONS_2302.csv"
PATH_2303 = "Datos\E01_T_DEU_CONS_2303.csv"
PATH_DICC = "Datos\Diccionario T_DEU_CONS.xlsx"
BANCO_CODS = [39,27]
MONEDA_CODS = [1,2,3] # Peso chileno, UF y dolar

In [16]:
# DICC_CODFAM = list(pd.read_excel(PATH_DICC, sheet_name='CODFAM')['CODIGO'])[1:]
# DICC_CODTLP = list(pd.read_excel(PATH_DICC, sheet_name='CODTLP')['CODIGO'])
# DICC_PROD = list(pd.read_excel(PATH_DICC, sheet_name='CODPRO')['CODIGO'])[1:]

# A futuro definimos esto segun columnas
umbral = {'R': 5}

# Lectura de datos

In [ ]:
dfdict = pd.read_excel('E01_T_DEU_CONS/Diccionario T_DEU_CONS.xlsx')
df2210 = pd.read_csv(PATH_2210, delimiter=";")
df2211 = pd.read_csv(PATH_2211,delimiter=";")
df2212 = pd.read_csv(PATH_2212,delimiter=";")
df2301 = pd.read_csv(PATH_2301,delimiter=";")
df2302 = pd.read_csv(PATH_2302,delimiter=";")
df2303 = pd.read_csv(PATH_2303,delimiter=";")

# Tests

In [10]:
def to_float(s):
    if type(s) != float:
        s = str(s).split(",")
        if len(s) == 1:
            return float(f"{int(s[0])}.0")
        return float(f"{int(s[0])}.{int(s[-1])}")
    return s

## Formato

In [12]:
def formato(df):
    'Retorna alarmas si columnas de la df estan en mal formato: (formato actual, formato en diccionario)'

    dfdic = pd.read_excel(PATH_DICC, \
                        sheet_name='T_DEU_CONS')[['Column_name', 'Type', 'Length']]
    dt = df.dtypes

    alertas = []
    for i in dfdic.index:

        if ('int' in dfdic['Type'][i]) and (dt[dfdic['Column_name'][i]] == np.int64):
            pass # formato correcto
        elif ('char' in dfdic['Type'][i]) and (dt[dfdic['Column_name'][i]] == str):
            pass # formato correcto
        elif ('char' in dfdic['Type'][i]) and (dt[dfdic['Column_name'][i]] == object):
            pass # formato correcto (Consideramos validos char como objetos)
        elif ('float' in dfdic['Type'][i]) and (dt[dfdic['Column_name'][i]] == float):
            pass # formato correcto
        elif ('decimal' in dfdic['Type'][i]) and (dt[dfdic['Column_name'][i]] == float):
            pass # formato correcto
        else:
            alertas.append(f'ALERTA ({dfdic["Column_name"][i]}): ({dt[dfdic["Column_name"][i]]}, {dfdic["Type"][i]})')

    return alertas

## Duplicidad

In [17]:
def duplicidad_id_mov(df, umbral = umbral['R']):
    'Revisa si el indice unico (Rut, NUMOPE) es unico, y retorna si cumple o no y el porcentaje de error'
    df1 = df.groupby(['RUT', 'NUMOPE']).size().reset_index(name='Size')
    num = int(df1[df1.Size > 1].shape[0])
    den = int(df.shape[0])
    
    if 100*(num/den) > umbral:
        return "No cumple", 100*(num/den)
    return "No cumple", 100*(num/den)

In [18]:
def duplicidad_num_cuenta(df, umbral = umbral['R']):
    'Revisa si hay duplicados, y retorna si cumple o no y el porcentaje de error'
    df1 = df.loc[:, ['RUT', 'NUMOPE']]
    Ob = df1.duplicated().value_counts()
    op_duplicadas = Ob[True]
    den = int(df.shape[0])
    if op_duplicadas == 0:
        return "Cumple",0
    return "No Cumple",100*(op_duplicadas/den)

## Vigencia

In [122]:
def fecha(s):
    "Recibe string fecha (int o float), retorna True si NO es fecha valida"
    if s <= 0:
        return True
    if type(s) == float:
        return int(str(s).split(".")[0][-2:]) > 31
    elif type(s) == int:
        return int(str(s)[-2:])> 31

def fechas_validas(df, umbral = umbral['R']):
    'Revisa si los dias en FECOPE estan en formato correcto, y retorna si cumple o no y el porcentaje de error'
    'Acepta solo formato int o float, de otra manera rechaza en totalidad'
    if df.FECOPE.dtype != np.int64 :
        return "No cumple", 100, ('Formato invalido')
    cantidad = df[~df.FECOPE.isna()].FECOPE.apply(fecha).value_counts() # si es float
    print(cantidad)
    print(df[~df.FECOPE.isna()].shape)
    prc = 100*(1-cantidad[False]/df[~df.FECOPE.isna()].shape[0])
    if prc > umbral:
        return "No cumple", prc
    return "Cumple", prc

In [20]:
def fec_movimiento(df, umbral = umbral['R']):
    'Revisa los NaN en las fechas, y retorna si cumple o no y el porcentaje de error'
    if df.FECOPE.dtype == int:
        a = "formato correcto"
    else:
        a="formato incorrecto"
    num1 = int(df.TASAINT.shape[0])
    den1 = int(df.TASAINT.isna().sum())

    if 100*(den1/num1) > umbral:
        return "No cumple", 100*(den1/num1)
    return "Cumple", 100*(den1/num1)

## Razonabilidad

In [21]:
#8 EXACTITUD PARA INTERES
def exactitud_intereses(df, umbral = umbral['R']):
    'Revisa la proporcion de NaN en TASAINT, y retorna si cumple o no y el porcentaje de error'
    if df.TASAINT.dtype == float:
        a = "formato correcto"
    else:
        a="formato incorrecto"
    num = int(df.TASAINT.shape[0])
    den = int(df.TASAINT.isna().sum())
    if 100*(den/num) > umbral:
        return "No cumple", 100*(den/num)
    return "Cumple", 100*(den/num)

## Exactitud

In [22]:
# Test de MTOREV 15/16
def test_mtorev(df, umbral = umbral['R']):
    'Revisa si MTOREV toma valores 0, y retorna si cumple o no y el porcentaje de error'
    n = df.shape[0]
    if df.MTOREV.dtype == float:
        prc = 100*df[df.MTOREV == 0].shape[0]/n
    else:
        df['MTOREVf'] = df.MTOREV.apply(to_float)
        prc = 100*df[df.MTOREVf == 0].shape[0]/n
    if prc > umbral:
            return ("No cumple", prc)

In [23]:
# Test de ESTADO 17
def test_estado(df):
    ### PUEDE SER QUE ESTE TEST ESTE MALO? EN NUESTRA BASE DE DATOS HAY MAS VALORES DE ESTADO ###
    '?, y retorna si cumple o no y el porcentaje de error'
    count = len(df) - len(df[(df["ESTADO"].isin([1,2,3,4]))])
    if count > 0:
        return "No Cumple", 100*(count/len(df))
    else:
        "Cumple",0

## Integridad

In [52]:
# Test RUT 19
# No se puede hacer

In [53]:
# Test COD_TRANSACCION 20
# No se que es

In [24]:
def cod_banco(df,codigos_banc_val):
    'Revisa que el codigo de banco este entre los valores definidos, y retorna si cumple o no y el porcentaje de error'
    if   df["BANCO"].isin(codigos_banc_val).all():
        return "Cumple",0
    
    return "No Cumple",100*(len(df["BANCO"].isin(codigos_banc_val))/len(df))

In [25]:
def cod_divisa(dataframe,codigos_moneda_val):
    'Verificar que las monedas esten dentro de las definidas'
    if   dataframe["MONEDA"].isin(codigos_moneda_val).all():
        return "Cumple",0
    
    return "No Cumple",100*(len(dataframe[(dataframe["MONEDA"].isin(codigos_moneda_val))]))/(len(dataframe))

In [26]:
def cod_producto(dataframe, umbral = umbral['R']):
    'Verifica que CODPRO este dentro de los valores definidos'
    count = dataframe[~dataframe.CODPRO.isin(DICC_PROD)].shape[0]
    if count/dataframe.shape[0] > umbral:
        return "No Cumple", 100*count/dataframe.shape[0]
    return "Cumple", 0

In [27]:
def cod_tlp(dataframe, umbral = 5):
    'Verificar que CODTLP este dentro de los valores especificados'
    count = dataframe[~dataframe.CODPRO.isin(DICC_CODTLP)].shape[0]
    if count/dataframe.shape[0] > umbral:
        return "No Cumple", 100*count/dataframe.shape[0]
    return "Cumple", 0

## Consistencia

In [57]:
# Test 26:  No tenemos acceso al codigo de finalizacion del contrato
# En general, no tenemos la info para implementar los tests de consistencia

In [28]:
def DIASMORA0_ESTADO1(df, umbral = 5):
    'Test (DIASMORA0_ESTADO1): Revisar que si DIASMORA es 0, entonces ESTADO = 1'
    
    # Numero de filas con DIASMORA = 0
    n = df[df.DIASMORA == 0].shape[0]

    # Si los dias de mora son 0, las cuotas deberian estar vigentes (solo estado 1)
    distr = df[(df.DIASMORA == 0)].ESTADO.value_counts()
    prc = (1 - distr[1]/n)*100
    
    if prc > umbral:
        return ('No cumple', round(prc,2))
    return ('Cumple', round(prc,2))

In [29]:
def DIASMORA30_ESTADO2(df, umbral = 5):
    '''Test (DIASMORA30_ESTADO2):  
    Si los dias de mora son < 30 y > 0, entonces solo debemos obtener estado: 2 (0 < mora < 30 dias)'''

    # Numero de filas con DIASMORA > 0 y < 30
    n = df[(df.DIASMORA < 30) & (df.DIASMORA > 0)].shape[0]

    # Si los dias de mora son < 30, las cuotas deberian estar en mora 1 (solo estado 2)
    distr = df[(df.DIASMORA < 30) & (df.DIASMORA > 0)].ESTADO.value_counts()
    prc = (1 - distr[[1,2]].sum()/n)*100
    
    if prc > umbral:
        return ('No cumple', round(prc,2))
    return ('Cumple', round(prc,2))

In [30]:
# No tiene sentido que aparezcan con estado 1 ni 2 ya que los DIASMORA son mayores a 30
def DIASMORA89_ESTADO5(df, umbral = 5):
    '''Test (DIASMORA89_ESTADO5):  
    Si los dias de mora son <= 89 y >= 30, entonces solo debemos obtener estado: 5'''

    # Numero de filas con DIASMORA >= 30 y <= 89
    n = df[(df.DIASMORA >= 30) & (df.DIASMORA <= 89)].shape[0]

    #  Si los dias de mora son (<= 89 y >= 30), entonces solo debemos obtener estado: 5
    distr = df[(df.DIASMORA >= 30) & (df.DIASMORA <= 89)].ESTADO.value_counts()
    prc = (1 - distr[5]/n)*100
    
    if prc > umbral:
        return ('No cumple', round(prc,2))
    return ('Cumple', round(prc,2))

In [31]:
def test_and_write(df, FILENAME):
    'Ejecuta los tests anteriores y escribe los resultados en FILENAME.csv'
    tests = pd.DataFrame(columns=['Test', 'Resultado'])
    tests.loc[len(tests)] = ['Formato', formato(df)]
    tests.loc[len(tests)] = ['duplicidad_id_mov', duplicidad_id_mov(df)]
    tests.loc[len(tests)] = ['duplicidad_num_cuenta', duplicidad_num_cuenta(df)]
    tests.loc[len(tests)] = ['fechas_validas', fechas_validas(df)]
    tests.loc[len(tests)] = ['fec_movimiento', fec_movimiento(df)]
    tests.loc[len(tests)] = ['exactitud_intereses', exactitud_intereses(df)]
    tests.loc[len(tests)] = ['test_mtorev', test_mtorev(df)]
    tests.loc[len(tests)] = ['test_estado', test_estado(df)]
    tests.loc[len(tests)] = ['cod_banco', cod_banco(df,BANCO_CODS)]
    tests.loc[len(tests)] = ['cod_divisa', cod_divisa(df,MONEDA_CODS)]
    tests.loc[len(tests)] = ['cod_producto', cod_producto(df)]
    tests.loc[len(tests)] = ['cod_tlp', cod_tlp(df)]
    tests.loc[len(tests)] = ['DIASMORA0_ESTADO1', DIASMORA0_ESTADO1(df)]
    tests.loc[len(tests)] = ['DIASMORA30_ESTADO2', DIASMORA30_ESTADO2(df)]
    tests.loc[len(tests)] = ['DIASMORA89_ESTADO5', DIASMORA89_ESTADO5(df)]
    tests.to_csv(FILENAME, index=False)

    return tests

# 2210

In [ ]:
df2210 = pd.read_csv(PATH_2210, delimiter=";")

In [77]:
formato(df2210)

ALERTA (OPENOIBS): (int64, varchar)
ALERTA (MONEDA): (float64, char)
ALERTA (DIASMORA): (float64, smallint)
ALERTA (TIPDEUDA): (float64, smallint)
ALERTA (CUOTOT): (float64, smallint)
ALERTA (CUOIMP): (float64, smallint)
ALERTA (CUOPEN): (float64, smallint)
ALERTA (FECOPE): (float64, int)
ALERTA (FECVEN): (float64, int)
ALERTA (MARDET): (float64, char)
ALERTA (SALDOPUNTA): (int64, float)
ALERTA (PERIODICIDAD): (float64, smallint)
ALERTA (FECACT): (float64, int)
ALERTA (FECBAJ): (float64, int)
ALERTA (MARDETF): (float64, smallint)
ALERTA (FPVENC): (float64, int)
ALERTA (GASTO): (float64, bigint)


In [86]:
duplicidad_id_mov(df2210)

La cantidad de llaves unicas repetidas es de es de 0.0003613309408407304 %
La cantidad de NUMOPE que cumplen el formato es de  100.0 %


<ipython-input-8-67892d174b63>:9: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  print(f"La cantidad de NUMOPE que cumplen el formato es de  {100*float(int(df.NUMOPE.apply(lambda x: len(str(x)) <= 8).value_counts()))/den} %")


In [12]:
duplicidad_num_cuenta(df2210)

La cantidad de filas duplicadas con respecto a 'RUT', 'NUMOPE' es de 0.0003613309408407304 %


('No Cumple', 0.0003613309408407304)

In [14]:
fec_movimiento(df2210)

La cantidad de fechas con valor entre 0 y 31 dias son 251947100 y representan un  91.03628268509358 % de la base total, incluyendo valores duplicados
La cantidad de fechas con valor nulo son 248075 y representan un  8.96371731490642 % de la base total, incluyendo valores duplicados
La cantidad de datos nan en FECOPE es de 6.115020310412184 %, los datos estan en formato incorrecto


<ipython-input-13-38730266fab7>:19: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  print(f"La cantidad de fechas con valor entre 0 y 31 dias son {100*int(cantidad)} y representan un  {100*(int(cantidad)/den)} % de la base total, incluyendo valores duplicados")


In [17]:
exactitud_intereses(df2210)

La cantidad de datos nan en TASAINT es de 6.115020310412184 %, los datos estan en formato correcto


In [65]:
test_mtorev(df2210)

('No cumple', 87.31609158438559)

In [19]:
test_estado(df2210)

('No Cumple', 28.577194380870274)

In [26]:
cod_banco(df2210,BANCO_CODS)

('Cumple', 0)

In [27]:
cod_divisa(df2210,MONEDA_CODS)

('No Cumple', 70.37613828279639)

In [38]:
cod_producto(df2210)

('Cumple', 0)

In [74]:
cod_tlp(df2210)

('Cumple', 0)

In [78]:
DIASMORA0_ESTADO1(df2210)

('No cumple', 11.96)

In [81]:
DIASMORA30_ESTADO2(df2210)

('Cumple', 1.85)

In [85]:
DIASMORA89_ESTADO5(df2210)

('No cumple', 15.17)

In [154]:
t2210 = test_and_write(df2210)
t2210.to_csv('tests2210.csv', index=False)

La cantidad de filas duplicadas con respecto a 'RUT', 'NUMOPE' es de 0.0003613309408407304 %


# 2211

In [30]:
t2211 = test_and_write(df2211, 'tests2211.csv')

La cantidad de filas duplicadas con respecto a 'RUT', 'NUMOPE' es de 0.0004304506531371244 %


# 2212

In [35]:
t2212 = test_and_write(df2212, 'tests2212.csv')

La cantidad de filas duplicadas con respecto a 'RUT', 'NUMOPE' es de 0.00042563137981535407 %


# 2301

In [36]:
t2301 = test_and_write(df2301, 'tests2301.csv')

La cantidad de filas duplicadas con respecto a 'RUT', 'NUMOPE' es de 0.00038851822680281283 %


# 2302

In [38]:
t2302 = test_and_write(df2302, 'tests2302.csv')

La cantidad de filas duplicadas con respecto a 'RUT', 'NUMOPE' es de 0.00028132145125297055 %


# 2303

In [ ]:
t2303 = test_and_write(df2303, 'tests2303.csv')

# En REV

In [125]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PATH_2210 = "..\..\Datos\E01_T_DEU_CONS_2210_REV.csv"
PATH_2211 = "Datos\E01_T_DEU_CONS_2211_REV.csv"
PATH_2212 = "Datos\E01_T_DEU_CONS_2212_REV.csv"
PATH_2301 = "Datos\E01_T_DEU_CONS_2301_REV.csv"
PATH_2302 = "Datos\E01_T_DEU_CONS_2302_REV.csv"
PATH_2303 = "Datos\E01_T_DEU_CONS_2303_REV.csv"
PATH_DICC = "..\..\Datos\Diccionario T_DEU_CONS.xlsx"
BANCO_CODS = [39,27]
MONEDA_CODS = [1,2,3] # Peso chileno, UF y dolar

DICC_CODFAM = list(pd.read_excel(PATH_DICC, sheet_name='CODFAM')['CODIGO'])[1:]
DICC_CODTLP = list(pd.read_excel(PATH_DICC, sheet_name='CODTLP')['CODIGO'])
DICC_PROD = list(pd.read_excel(PATH_DICC, sheet_name='CODPRO')['CODIGO'])[1:]

In [9]:
#dfdict = pd.read_excel('E01_T_DEU_CONS/Diccionario T_DEU_CONS.xlsx')
df2210 = pd.read_csv(PATH_2210, delimiter=";")
# df2211 = pd.read_csv(PATH_2211,delimiter=";")
# df2212 = pd.read_csv(PATH_2212,delimiter=";")
# df2301 = pd.read_csv(PATH_2301,delimiter=";")
# df2302 = pd.read_csv(PATH_2302,delimiter=";")
# df2303 = pd.read_csv(PATH_2303,delimiter=";")

In [126]:
test_and_write(df2210, 'tests2210REV.csv')

FECOPE
False    2519469
True      248077
Name: count, dtype: int64
(2767546, 54)


,Test,Resultado
0,Formato,"[ALERTA (OPENOIBS): (int64, varchar), ALERTA (..."
1,duplicidad_id_mov,"(No cumple, 0.0003613309408407304)"
2,duplicidad_num_cuenta,"(No Cumple, 0.0003613309408407304)"
3,fechas_validas,"(No cumple, 8.963789581094584)"
4,fec_movimiento,"(No cumple, 6.115020310412184)"
5,exactitud_intereses,"(No cumple, 6.115020310412184)"
6,test_mtorev,"(No cumple, 87.31609158438559)"
7,test_estado,"(No Cumple, 28.577194380870274)"
8,cod_banco,"(Cumple, 0)"
9,cod_divisa,"(No Cumple, 94.63535565443176)"
